In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from skimage import color
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# Reading .dcm file and returns a numpy array of the imaging data

def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    if ds.Modality != 'DX' or ds.BodyPartExamined not in ('CHEST', 'RIBCAGE', 'LUNG', 'CHESTABDOMEN'):
        print('*******************')
        print('This is not a chest xray')
        print('*******************\n')
        return None
    img = ds.pixel_array
    return img

In [3]:
def check_pneumonia(filename): 
    print('Checking pneumonia in header {} ...'.format(filename))
    ds = pydicom.dcmread(filename)     
    pneumonia = 'pneumonia'  in ds.StudyDescription.lower()
    return pneumonia

In [4]:
# Preprocessing  of the imaging data

def preprocess_image(filename, img, model_input_size):
    print('Preprocessing image ...')
    proc_img = resize(img, (model_input_size[1], model_input_size[2])) #resizing
    proc_img = proc_img/255 # Normalization
    
    if pydicom.dcmread(filename).PhotometricInterpretation == 'MONOCHROME2': 
        proc_img = color.grey2rgb(proc_img) #converting to rgb
        proc_img = proc_img.reshape(model_input_size) 
        
    if pydicom.dcmread(filename).PhotometricInterpretation == 'MONOCHROME1':
        proc_img = 1-proc_img # inverting color
        proc_img = color.grey2rgb(proc_img) #converting to rgb
        proc_img = proc_img.reshape(model_input_size)    

    return proc_img

In [5]:
# Loading trained model with weights and compiles it 

def load_model(model_path, weight_path):
    print('loading model')
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    print('Loaded model from disk')
    
    return model

In [6]:
# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    print('Predicting label ...')
    prediction = model.predict(img, verbose = True)    
    prediction = 0 if prediction < thresh else 1
    return prediction

In [7]:
def decoder(boolean):
    decoded = 'pneumonia' if boolean else 'non-pneumonia'
    return decoded

In [8]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = './xray_class_my_model.best.hdf5'

model_input_size=(1,224,224,3)

my_model = load_model(model_path, weight_path)
thresh = 0.44 # value giving maximum f score during the training 

loading model
Loaded model from disk


In [9]:
# use the .dcm files to test predictions
for filename in test_dicoms:
    
    img = np.array([])
    img = check_dicom(filename)
    pydicom.dcmread(filename)
    if img is None :
        continue
    img_proc = preprocess_image(filename,img,model_input_size)   
    pneumonia = check_pneumonia(filename)
    pred = predict_image(my_model,img_proc,thresh)
    print('*******************')
    print('This {} case is predicted as {}'.format(decoder(pneumonia), decoder(pred)))
    print('*******************\n')

Load file test1.dcm ...
Preprocessing image ...
Checking pneumonia in header test1.dcm ...
Predicting label ...
1/1 [==============================] - 1s 888ms/step
*******************
This non-pneumonia case is predicted as non-pneumonia
*******************

Load file test2.dcm ...
Preprocessing image ...
Checking pneumonia in header test2.dcm ...
Predicting label ...
1/1 [==============================] - 1s 604ms/step
*******************
This non-pneumonia case is predicted as non-pneumonia
*******************

Load file test3.dcm ...
Preprocessing image ...
Checking pneumonia in header test3.dcm ...
Predicting label ...
1/1 [==============================] - 1s 586ms/step
*******************
This non-pneumonia case is predicted as non-pneumonia
*******************

Load file test4.dcm ...
Preprocessing image ...
Checking pneumonia in header test4.dcm ...
Predicting label ...
1/1 [==============================] - 1s 602ms/step
*******************
This non-pneumonia case is predicte